<a href="https://colab.research.google.com/github/booluckgmie/training/blob/main/python/PortalInfoBanjir_TaburanHujan_ArasAir_(Daily).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Identify list of States

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

# HTML code
html = '''
<select id="state" name="state" onchange="searchDistrict();" class="form-control input-md">
  <option value="PLS" selected="selected">Perlis  </option>
  <option value="KDH">Kedah</option>
  <option value="PNG">Pulau Pinang  </option>
  <option value="PRK">Perak  </option>
  <option value="SEL">Selangor  </option>
  <option value="WLH">Wilayah Persekutuan Kuala Lumpur  </option>
  <option value="PTJ">Wilayah Persekutuan Putrajaya  </option>
  <option value="NSN">Negeri Sembilan   </option>
  <option value="MLK">Melaka   </option>
  <option value="JHR">Johor   </option>
  <option value="PHG">Pahang   </option>
  <option value="TRG">Terengganu   </option>
  <option value="KEL">Kelantan   </option>
  <option value="SRK">Sarawak   </option>
  <option value="SAB">Sabah    </option>
  <option value="WLP">Wilayah Persekutuan Labuan     </option>                            
</select>
'''

# Parse HTML code
soup = BeautifulSoup(html, 'html.parser')

# Extract select tag
select_tag = soup.find('select', id='state')

# Extract option tags
option_tags = select_tag.find_all('option')

# Create list of dictionaries
data = []
for option_tag in option_tags:
    statecode = option_tag['value']
    StateName = option_tag.text.strip()
    data.append({'statecode': statecode, 'state': StateName})

# Create dataframe
df2 = pd.DataFrame(data)

df2


,statecode,state
0,PLS,Perlis
1,KDH,Kedah
2,PNG,Pulau Pinang
3,PRK,Perak
4,SEL,Selangor
5,WLH,Wilayah Persekutuan Kuala Lumpur
6,PTJ,Wilayah Persekutuan Putrajaya
7,NSN,Negeri Sembilan
8,MLK,Melaka
9,JHR,Johor


## Loop Scrape Rainfall Data by State

In [ ]:
# Define the state codes and state names
state_codes = df2['statecode'].tolist()
state_names = df2['state'].tolist()

# Create empty list to store dataframes
df_list = []

# Loop through each state code and extract data
for code, name in zip(state_codes, state_names):
    # Make the request for the current state code
    conn.request("GET", f"/wp-content/themes/shapely/agency/searchresultrainfall.php?state={code}&district=ALL&station=ALL&loginStatus=0&language=0", payload, headers)
    res = conn.getresponse()
    data = res.read()

    # Parse the HTML response
    soup = BeautifulSoup(data, 'html.parser')
    table = soup.find("table", attrs={"class": "normaltable", "id": "normaltable1"})
    rows = table.find_all("td")

    # Create empty lists to store data
    data_list = []
    temp_list = []

    # Extract data from each row and append to list
    for row in rows:
        temp_list.append(row.get_text())
        if len(temp_list) == 13:
            data_list.append(temp_list)
            temp_list = []

    # Convert list to dataframe and append to list
    df_state = pd.DataFrame(data_list, columns=['No', 'StationID', 'NamaStesen', 'Daerah', 'KemaskiniTerakhir', 'past6day', 'past5day', 'past4day', 'past3day', 'past2day', 'past1day', 'today', 'Jumlah1Jam(Terkini)'])
    df_state['statecode'] = code
    df_state['StateName'] = name
    df_list.append(df_state)

# Concatenate dataframes and reset index
df = pd.concat(df_list).reset_index(drop=True)

# Print dataframe
df

,No,StationID,NamaStesen,Daerah,KemaskiniTerakhir,past6day,past5day,past4day,past3day,past2day,past1day,today,Jumlah1Jam(Terkini),statecode,StateName
0,1,6601001,Wang Kelian,Padang Besar,10/04/2023 14:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0,PLS,Perlis
1,2,6602002,Kaki Bukit,Padang Besar,10/04/2023 14:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,1.0,0.0,PLS,Perlis
2,3,6603002,Padang Besar,Padang Besar,10/04/2023 14:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0,PLS,Perlis
3,4,6602006,Kg. Chantik,Padang Besar,10/04/2023 14:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0,PLS,Perlis
4,5,6502011,Empangan Timah Tasuh,Padang Besar,10/04/2023 14:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0,PLS,Perlis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,6,5252101,Kinabenua,Labuan,10/04/2023 15:00:00,49.5,0.0,10.0,0.0,1.5,0.0,8.0,0.0,WLP,Wilayah Persekutuan Labuan
979,7,5352103,Ganggarak,Labuan,10/04/2023 15:00:00,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WLP,Wilayah Persekutuan Labuan
980,8,5255105,Waterfront D/S,Labuan,10/04/2023 15:00:00,67.5,0.0,2.5,0.0,0.0,0.0,0.0,0.0,WLP,Wilayah Persekutuan Labuan
981,9,5351001,Sg. Lada,Labuan,10/04/2023 15:00:00,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WLP,Wilayah Persekutuan Labuan


### Kategori Keamatan Hujan (dalam satu jam)
Renyai	1-10 (mm);
Sederhana	11-30 (mm);
Lebat	31-60 (mm);
Sangat Lebat	>60 (mm)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   No                   983 non-null    object
 1   StationID            983 non-null    object
 2   NamaStesen           983 non-null    object
 3   Daerah               983 non-null    object
 4   KemaskiniTerakhir    983 non-null    object
 5   past6day             983 non-null    object
 6   past5day             983 non-null    object
 7   past4day             983 non-null    object
 8   past3day             983 non-null    object
 9   past2day             983 non-null    object
 10  past1day             983 non-null    object
 11  today                983 non-null    object
 12  Jumlah1Jam(Terkini)  983 non-null    object
 13  statecode            983 non-null    object
 14  StateName            983 non-null    object
dtypes: object(15)
memory usage: 115.3+ KB


### checking

In [ ]:
df[df['statecode']== 'SEL']

,No,StationID,NamaStesen,Daerah,KemaskiniTerakhir,past6day,past5day,past4day,past3day,past2day,past1day,today,Jumlah1Jam(Terkini),statecode,StateName
212,1,3717101,Bukit Fraser,Hulu Selangor,10/04/2023 15:00:00,2.5,1.0,-9999.0,14.0,3.5,0.5,0.0,0.0,SEL,Selangor
213,2,3517022,Kg. Pertak,Hulu Selangor,10/04/2023 14:30:00,0.5,10.5,0.0,2.0,0.0,0.0,0.0,0.0,SEL,Selangor
214,3,3516022,Loji Air KKB,Hulu Selangor,10/04/2023 15:00:00,0.5,1.0,-9999.0,0.0,0.5,0.0,0.0,0.0,SEL,Selangor
215,4,3516026,Kerling,Hulu Selangor,10/04/2023 15:00:00,0.0,0.0,-9999.0,0.0,10.0,0.5,0.0,0.0,SEL,Selangor
216,5,3516025,Ampang Pecah,Hulu Selangor,10/04/2023 14:45:00,0.0,0.0,-9999.0,0.0,0.0,0.0,0.0,0.0,SEL,Selangor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,120,2914001,RS Batu 8,Kuala Langat,10/04/2023 14:15:00,23.5,3.5,19.5,8.0,3.0,10.0,4.0,0.5,SEL,Selangor
332,121,2615180,Kg. Kundang,Kuala Langat,10/04/2023 15:00:00,0.0,0.0,-9999.0,0.0,12.5,0.0,0.0,0.0,SEL,Selangor
333,122,0241301RF,Kg. Sesapan Batu Rembau,Hulu Langat,10/04/2023 15:00:00,0.5,0.0,-9999.0,52.0,1.0,0.5,0.0,0.0,SEL,Selangor
334,123,0232941RF,Kg. Kemensah,Gombak,10/04/2023 15:00:00,0.0,5.5,-9999.0,19.0,21.5,0.5,0.0,0.0,SEL,Selangor


In [ ]:
df.to_csv('thujan_state.csv', index=False)

In [ ]:
df.columns

Index(['No', 'StationID', 'NamaStesen', 'Daerah', 'KemaskiniTerakhir',
       'past6day', 'past5day', 'past4day', 'past3day', 'past2day', 'past1day',
       'today', 'Jumlah1Jam(Terkini)', 'statecode', 'StateName'],
      dtype='object')

In [ ]:
df[['No', 'StationID', 'NamaStesen', 'Daerah', 'KemaskiniTerakhir','today', 'statecode', 'StateName']]

,No,StationID,NamaStesen,Daerah,KemaskiniTerakhir,today,statecode,StateName
0,1,6601001,Wang Kelian,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
1,2,6602002,Kaki Bukit,Padang Besar,10/04/2023 14:00:00,1.0,PLS,Perlis
2,3,6603002,Padang Besar,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
3,4,6602006,Kg. Chantik,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
4,5,6502011,Empangan Timah Tasuh,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
...,...,...,...,...,...,...,...,...
978,6,5252101,Kinabenua,Labuan,10/04/2023 15:00:00,8.0,WLP,Wilayah Persekutuan Labuan
979,7,5352103,Ganggarak,Labuan,10/04/2023 15:00:00,0.0,WLP,Wilayah Persekutuan Labuan
980,8,5255105,Waterfront D/S,Labuan,10/04/2023 15:00:00,0.0,WLP,Wilayah Persekutuan Labuan
981,9,5351001,Sg. Lada,Labuan,10/04/2023 15:00:00,0.0,WLP,Wilayah Persekutuan Labuan


In [ ]:
daily = df[['No', 'StationID', 'NamaStesen', 'Daerah', 'KemaskiniTerakhir','today', 'statecode', 'StateName']]
daily

,No,StationID,NamaStesen,Daerah,KemaskiniTerakhir,today,statecode,StateName
0,1,6601001,Wang Kelian,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
1,2,6602002,Kaki Bukit,Padang Besar,10/04/2023 14:00:00,1.0,PLS,Perlis
2,3,6603002,Padang Besar,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
3,4,6602006,Kg. Chantik,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
4,5,6502011,Empangan Timah Tasuh,Padang Besar,10/04/2023 14:00:00,0.0,PLS,Perlis
...,...,...,...,...,...,...,...,...
978,6,5252101,Kinabenua,Labuan,10/04/2023 15:00:00,8.0,WLP,Wilayah Persekutuan Labuan
979,7,5352103,Ganggarak,Labuan,10/04/2023 15:00:00,0.0,WLP,Wilayah Persekutuan Labuan
980,8,5255105,Waterfront D/S,Labuan,10/04/2023 15:00:00,0.0,WLP,Wilayah Persekutuan Labuan
981,9,5351001,Sg. Lada,Labuan,10/04/2023 15:00:00,0.0,WLP,Wilayah Persekutuan Labuan


In [ ]:
daily['today'] = daily['today'].apply(pd.to_numeric, errors='coerce')

C:\Users\najmi.ariffin\AppData\Local\Temp\2\ipykernel_67140\3361599962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily['today'] = daily['today'].apply(pd.to_numeric, errors='coerce')


In [ ]:
daily[daily['today'] > 11]

,No,StationID,NamaStesen,Daerah,KemaskiniTerakhir,today,statecode,StateName
224,13,3714080,Kg. Sg. Selisek,Hulu Selangor,10/04/2023 14:45:00,33.0,SEL,Selangor
301,90,3017109,Sg. Kantan,Hulu Langat,10/04/2023 11:00:00,17.0,SEL,Selangor
393,31,2820011,Kg. Ulu Klawang (1),Jelebu,10/04/2023 15:00:00,23.0,NSN,Negeri Sembilan
459,14,2330002,Kg. Gatom,Segamat,10/04/2023 15:00:00,31.0,JHR,Johor
460,15,2330009,LPP Labis,Segamat,10/04/2023 15:00:00,67.0,JHR,Johor
464,19,2231001,Ladang Chan Wing,Segamat,10/04/2023 15:00:00,13.0,JHR,Johor
883,181,3330001,Sungai Sebatang,Bintulu,10/04/2023 14:30:00,64.0,SRK,Sarawak
902,200,3939046,Sungai Sibuti,Miri,10/04/2023 15:00:00,17.5,SRK,Sarawak
922,220,3547001,Long Luteng,Miri,10/04/2023 14:45:00,51.0,SRK,Sarawak
931,229,3941001,Beluru,Miri,10/04/2023 14:45:00,33.5,SRK,Sarawak


## Single State

In [ ]:
import http.client
from bs4 import BeautifulSoup
import pandas as pd

conn = http.client.HTTPSConnection("publicinfobanjir.water.gov.my")
payload = ''
headers = {
'Cookie': 'qtrans_front_language=en'
}
conn.request("GET", "/wp-content/themes/shapely/agency/searchresultrainfall.php?state=PLS&district=ALL&station=ALL&loginStatus=0&language=0", payload, headers)
res = conn.getresponse()
data = res.read()

soup = BeautifulSoup(data, 'html.parser')

#Extract data from the table
table = soup.find("table", attrs={"class": "normaltable", "id": "normaltable1"})
rows = table.find_all("td")

# Create empty lists to store data
data_list = []
temp_list = []

# Extract data from each row and append to list
for row in rows:
    temp_list.append(row.get_text())
    if len(temp_list) == 13:
        data_list.append(temp_list)
        temp_list = []

# Convert list to dataframe
df = pd.DataFrame(data_list, columns=['No', 'StationID', 'NamaStesen', 'Daerah', 'KemaskiniTerakhir', 'Harian04/04/2023', 'Harian05/04/2023', 'Harian06/04/2023', 'Harian07/04/2023', 'Harian08/04/2023', 'Harian09/04/2023', 'TaburanHujanDariTengahMalam(10/04/2023)', 'Jumlah1Jam(Terkini)'])

# Print dataframe
df

,No,StationID,NamaStesen,Daerah,KemaskiniTerakhir,Harian04/04/2023,Harian05/04/2023,Harian06/04/2023,Harian07/04/2023,Harian08/04/2023,Harian09/04/2023,TaburanHujanDariTengahMalam(10/04/2023),Jumlah1Jam(Terkini)
0,1,6601001,Wang Kelian,Padang Besar,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0
1,2,6602002,Kaki Bukit,Padang Besar,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,1.0,0.0
2,3,6603002,Padang Besar,Padang Besar,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0
3,4,6602006,Kg. Chantik,Padang Besar,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0
4,5,6502011,Empangan Timah Tasuh,Padang Besar,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0
5,6,0010151RF,Bukit Temiang (F2),Padang Besar,10/04/2023 14:00:00,0.0,3.5,4.5,3.0,0.0,0.0,0.0,0.0
6,7,6502012,Kg. Batu Bertangkup,Arau,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,1.0,0.0,0.0
7,8,0010611RF,Sungai Gial di Kg Gial,Arau,10/04/2023 14:00:00,0.0,13.5,4.5,0.0,0.0,0.0,0.0,0.0
8,9,6402008,Ngolang di Perlis,Arau,10/04/2023 14:00:00,0.0,6.0,4.0,0.5,0.0,0.0,0.0,0.0
9,10,6402303,Arau,Arau,10/04/2023 13:00:00,0.0,0.0,-9999.0,-9999.0,-9999.0,0.0,0.0,0.0


## Data Paras Air

### single State

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://publicinfobanjir.water.gov.my/aras-air/data-paras-air/aras-air-data/?state=PLS&district=ALL&station=ALL"

response = requests.get(url, verify=False) 

soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", {"class": "normaltable-fluid"})
# table
headers = []
for th in table.find_all("th"):
    headers.append(th.text.strip())

rows = []
for tr in table.find_all("tr"):
    row = []
    for td in tr.find_all("td"):
        row.append(td.text.strip())
    if len(row) > 0:
        rows.append(row)

# Convert list to dataframe
df = pd.DataFrame(rows, columns=['id', 'station_id', 'station', 'district', 'river', 'parameter', 'datetime', 'aras air (m)', 'level_normal', 'level_waspada', 'level_amaran', 'level_bahaya'])

df


c:\Users\najmi.ariffin\Anaconda3\envs\ui\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'publicinfobanjir.water.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,id,station_id,station,district,river,parameter,datetime,aras air (m),level_normal,level_waspada,level_amaran,level_bahaya
0,1,6602402,Sg. Pelarit di Kaki Bukit,Padang Besar,Sungai Perlis,Sg. Pelarit,10/04/2023 16:00,35.13,35.60,38.30,38.60,39.00
1,2,6602404,Sg. Chuchoh di Kg. Chantik,Padang Besar,Sungai Perlis,Sg. Chuchoh,10/04/2023 16:00,36.52,35.50,37.50,37.80,38.50
2,3,6502436,Empangan Timah Tasuh,Padang Besar,Sungai Perlis,Empangan Timah Tasoh,10/04/2023 16:00,27.57,29.10,29.40,29.50,29.60
3,4,6502404,Sg. Buluh di Kg. Batu Bertangkup,Arau,Sungai Perlis,Sg. Buluh,10/04/2023 16:00,22.33,22.48,24.00,25.00,26.00
4,5,6502404,Sg. Buloh di Kg. Batu Bertangkup (F2),Arau,Sungai Perlis,Sg. Buluh,10/04/2023 16:00,22.53,22.48,24.00,25.00,26.00
5,6,6402411,Sg. Panggas Besar di Tok Biduk (F2),Arau,Sungai Perlis,Sg. Panggas Besar,10/04/2023 16:00,20.49,22.00,23.50,24.00,25.00
6,7,0010611RF,Sungai Gial di Kg. Gial,Arau,Sungai Perlis,Sg.Perlis Sub-basin,10/04/2023 16:00,6.98,8.00,8.50,8.80,9.40
7,8,6402401,Sg. Perlis di Kg. Sg. Bakau,Kangar,Sungai Perlis,Sg. Perlis,10/04/2023 16:00,1.91,1.50,3.00,3.30,3.50
8,9,6602404,Sg. Chuchoh di Kg. Chantik (F2),Padang Besar,Sungai Perlis,Sg. Chuchoh,10/04/2023 16:00,39.05,0.00,0.00,0.00,0.00
9,10,6502436,Empangan Timah Tasuh (F2),Padang Besar,Sungai Perlis,Empangan Timah Tasoh,10/04/2023 16:00,27.59,0.00,0.00,0.00,0.00


### Loop Scrape Data Paras Air by State

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of state codes to loop through
# state_codes = ['JHR', 'KDH', 'MLK', 'NSN', 'PHG', 'PLS', 'PRK', 'PNG', 'SBH', 'SWK', 'SGR', 'TRG', 'KUL', 'LBN', 'PJY']
# Define the state codes and state names
state_codes = df2['statecode'].tolist()
state_names = df2['state'].tolist()


# Empty list to store dataframes for each state
state_dfs = []

# Loop through state codes
for state_code in state_codes:
    url = f"https://publicinfobanjir.water.gov.my/aras-air/data-paras-air/aras-air-data/?state={state_code}&district=ALL&station=ALL"

    response = requests.get(url, verify=False) 

    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", {"class": "normaltable-fluid"})
    # table
    headers = []
    for th in table.find_all("th"):
        headers.append(th.text.strip())

    rows = []
    for tr in table.find_all("tr"):
        row = []
        for td in tr.find_all("td"):
            row.append(td.text.strip())
        if len(row) > 0:
            rows.append(row)

    # Convert list to dataframe
    df = pd.DataFrame(rows, columns=['id', 'station_id', 'station', 'district', 'river', 'parameter', 'datetime', 'aras air (m)', 'level_normal', 'level_waspada', 'level_amaran', 'level_bahaya'])
    
    # Add state code column to dataframe
    df['state'] = state_code
    
    # Append dataframe to list
    state_dfs.append(df)

# Concatenate dataframes for each state
result_df = pd.concat(state_dfs)

result_df


c:\Users\najmi.ariffin\Anaconda3\envs\ui\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'publicinfobanjir.water.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\najmi.ariffin\Anaconda3\envs\ui\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'publicinfobanjir.water.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\najmi.ariffin\Anaconda3\envs\ui\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'publicinfobanjir.water.gov.my'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.htm

,id,station_id,station,district,river,parameter,datetime,aras air (m),level_normal,level_waspada,level_amaran,level_bahaya,state
0,1,6602402,Sg. Pelarit di Kaki Bukit,Padang Besar,Sungai Perlis,Sg. Pelarit,10/04/2023 16:00,35.13,35.60,38.30,38.60,39.00,PLS
1,2,6602404,Sg. Chuchoh di Kg. Chantik,Padang Besar,Sungai Perlis,Sg. Chuchoh,10/04/2023 16:00,36.52,35.50,37.50,37.80,38.50,PLS
2,3,6502436,Empangan Timah Tasuh,Padang Besar,Sungai Perlis,Empangan Timah Tasoh,10/04/2023 16:00,27.57,29.10,29.40,29.50,29.60,PLS
3,4,6502404,Sg. Buluh di Kg. Batu Bertangkup,Arau,Sungai Perlis,Sg. Buluh,10/04/2023 16:00,22.33,22.48,24.00,25.00,26.00,PLS
4,5,6502404,Sg. Buloh di Kg. Batu Bertangkup (F2),Arau,Sungai Perlis,Sg. Buluh,10/04/2023 16:00,22.53,22.48,24.00,25.00,26.00,PLS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,0000281WL,Sg. Bangat di Kg. Sg. Bangat,Labuan,Sungai Kinabenua,Sg. Bangat,10/04/2023 16:00,0.54,0.60,1.00,1.80,2.20,WLP
1,2,0000241WL,Empangan Bukit Kuda,Labuan,Sungai Kinabenua,Sg. Kinabenua,10/04/2023 16:00,15.65,17.00,17.50,17.80,18.00,WLP
2,3,0000211WL,Waterfront D/S,Labuan,New Victoria,New Victoria,10/04/2023 16:00,0.21,0.60,1.00,1.25,1.50,WLP
3,4,0000261WL,Empangan Kerupang,Labuan,Sungai Kerupang,Sg. Kerupang,10/04/2023 16:00,16.09,16.05,16.90,17.30,17.50,WLP


### checking

In [ ]:
set(result_df['state'])

{'JHR',
 'KDH',
 'KEL',
 'MLK',
 'NSN',
 'PHG',
 'PLS',
 'PNG',
 'PRK',
 'SAB',
 'SEL',
 'SRK',
 'TRG',
 'WLH',
 'WLP'}

In [ ]:
result_df[result_df['state']== 'PRK']

,id,station_id,station,district,river,parameter,datetime,aras air (m),level_normal,level_waspada,level_amaran,level_bahaya,state
0,1,5108401,Sg. Ijok di Bekalan Ijok,Larut Matang dan Selama,Sungai Kerian,Sg. Ijok,10/04/2023 16:00,32.67,29.00,35.00,35.30,35.50,PRK
1,2,4907422,Sg. Kurau di Bt. 14 Batu Kurau,Larut Matang dan Selama,Sungai Kurau,Sg. Kurau,10/04/2023 16:00,19.82,23.50,24.00,24.70,25.40,PRK
2,3,5006401,Kolam Air Bukit Merah,Kerian,Sungai Kurau,Sg. Kurau,10/04/2023 16:00,8.55,8.68,9.00,9.04,9.14,PRK
3,4,5005405,Sg. Samagagah di Samagagah,Kerian,Sungai Kerian,Sg. Samagagah,10/04/2023 15:15,0.86,1.00,1.80,2.10,2.30,PRK
4,5,5513401,Tasik Temengor di Banding,Hulu Perak,Sungai Perak,Sg. Perak,10/04/2023 16:00,244.79,240.00,247.00,247.69,248.38,PRK
5,6,5411066,Sg. Perak di Kuala Kenderong,Hulu Perak,Sungai Perak,Sg. Perak,10/04/2023 16:00,110.83,110.00,111.80,111.85,111.90,PRK
6,7,4911445,Sg. Plus di Kg. Lintang,Kuala Kangsar,Sungai Perak,Sg. Pelus,10/04/2023 16:00,52.45,52.00,54.00,54.24,54.80,PRK
7,8,0181371WL,Pekan Manong (Sungai Perak),Kuala Kangsar,Sungai Perak,Sg. Perak,10/04/2023 16:00,0.33,0.00,0.00,0.00,0.00,PRK
8,9,0181351WL,Kg. Cik Zainal,Kinta,Sungai Perak,Sg. Kinta,10/04/2023 16:00,73.74,0.00,0.00,0.00,0.00,PRK
9,10,4613401,Empangan Ulu Kinta,Kinta,Sungai Perak,Sg. Kinta,10/04/2023 16:00,245.13,243.50,246.50,248.00,249.50,PRK
